# Multi Layer Perceptron (MLP) 

* Objectives: See the effect of nonlinearities, learn about regularization and dropout to combat overfitting.
* Prerequisites: Linear models (03.lin.ipynb), AutoGrad, Param, KnetArray, gpu, nll
* Knet: xavier, param, param0, relu, dropout (defined and explained)
* Knet: dir, gpu, minibatch, KnetArray (used by mnist.jl)
* Knet: SGD, train!, Training, load, save (used by trainresults)

In [ ]:
# Load data (see 02.mnist.ipynb)
using Knet: Knet, KnetArray, gpu, minibatch
include(Knet.dir("data","mnist.jl"))  # Load data
dtrn,dtst = mnistdata(xsize=(784,:)); # dtrn and dtst = [ (x1,y1), (x2,y2), ... ] where xi,yi are minibatches of 100

In [ ]:
(x,y) = first(dtst)
summary.((x,y))

In [ ]:
# For running experiments
using Knet: SGD, train!, Training
function trainresults(file,model,datasets...; optimizer=SGD(lr=0.1), whentorecord=0:600:60000, o...)
    if !isfile(file)
        t = Training(whentorecord,datasets...)
        train!(model,datasets[1];callback=t,optimizer=optimizer,o...)
        t.datasets = t.progress = nothing 
        Knet.save(file,"results",t)
    end
    t = Knet.load(file,"results")
    println(minimum.([t.losses..., t.errors...]))
    return t
end

## Model definition

In [ ]:
using AutoGrad: Param

In [ ]:
# Redefine Linear model (See 03.lin.ipynb):
struct Linear; w; b; end
(f::Linear)(x) = (f.w * x .+ f.b)

In [ ]:
# Linear(inputsize,outputsize) constructs a randomly initalized Linear model:
Linear(inputsize::Int,outputsize::Int) = Linear(param(outputsize,inputsize),param0(outputsize))
param(d...; init=xavier, atype=atype())=Param(atype(init(d...)))
param0(d...; atype=atype())=param(d...; init=zeros, atype=atype)
xavier(o,i) = (s = sqrt(2/(i+o)); 2s .* rand(o,i) .- s)
atype()=(gpu() >= 0 ? KnetArray{Float32} : Array{Float32})

In [ ]:
# MLP is a bunch of linear layers stuck together:
struct MLP; layers; end

In [ ]:
# MLP(x,h1,h2,...,hn,y): constructor for a n hidden layer MLP:
MLP(h::Int...)=MLP(Linear.(h[1:end-1], h[2:end]))

In [ ]:
# Here is an example model
model=MLP(784,64,10)
summary.(l.w for l in model.layers)

## Multiple linear layers do not improve over linear model

In [ ]:
ENV["COLUMNS"]=92

In [ ]:
# Let us try to just chain multiple linear layers
function (m::MLP)(x)
    for layer in m.layers
        x = layer(x)
    end
    return x
end

In [ ]:
# Train a linear model
@time lin1 = trainresults("lin1.jld2", Linear(784,10), dtrn, dtst); # 113s [0.241861, 0.266827, 0.0668667, 0.0747]

In [ ]:
# Train a multi-linear model
@time mlp1 = trainresults("mlp1.jld2", MLP(784,64,10), dtrn, dtst); # 110s [0.242123, 0.285064, 0.0697, 0.0799]

In [ ]:
# multilinear converges to a similar solution, not identical because problem is non-convex
using Plots
plot([lin1.losses[1], lin1.losses[2], mlp1.losses[1], mlp1.losses[2]], ylim=(0.0,0.4),
    labels=[:trnLin :tstLin :trnMulti :tstMulti],xlabel="Epochs",ylabel="Loss")  

In [ ]:
# error results also close to the linear model
plot([lin1.errors[1], lin1.errors[2], mlp1.errors[1], mlp1.errors[2]], ylim=(0.0,0.1),
    labels=[:trnLin,:tstLin,:trnMulti,:tstMulti], xlabel="Epochs", ylabel="Error")  

## Multiple linear layers are useless because they are equivalent to a single linear layer
If we write down what is being computed and do some algebra, we can show that what is being computed is still an affine function of the input, i.e. stacking multiple linear layers does not increase the representational capacity of the model:

\begin{align*}
\hat{p} &= \mbox{soft}(W_2 (W_1 x + b_1) + b_2) \\
&= \mbox{soft}((W_2 W_1)\, x + W_2 b_1 + b_2) \\
&= \mbox{soft}(W x + b)
\end{align*}

In [ ]:
Knet.gc() # to save gpu memory

## Multi Layer Perceptron (MLP) adds non-linearities between layers

In [ ]:
# Using nonlinearities between layers results in a model with higher capacity and helps underfitting
# relu(x)=max(0,x) is a popular function used for this purpose, it replaces all negative values with zeros.
using Knet: relu

In [ ]:
function (m::MLP)(x)
    for layer in m.layers
        x = layer(x)
        x = (layer == m.layers[end] ? x : relu.(x))  # <-- This one line makes a big difference
    end
    return x
end

In [ ]:
@time mlp2 = trainresults("mlp2.jld2", MLP(784,64,10), dtrn, dtst);  # 124s [0.00632429, 0.0888991, 0.00055, 0.0259]

## MLP solves underfitting but still has an overfitting problem

In [ ]:
# MLP solves the underfitting problem!  A more serious overfitting problem remains.
plot([lin1.losses[1], lin1.losses[2], mlp2.losses[1], mlp2.losses[2]], ylim=(0.0,0.4),
     labels=[:trnLin :tstLin :trnMLP :tstMLP],xlabel="Epochs",ylabel="Loss")

In [ ]:
# Test error improves from 7.5% to 2.3%!
plot([lin1.errors[1], lin1.errors[2], mlp2.errors[1], mlp2.errors[2]], ylim=(0.0,0.1),
    labels=[:trnLin,:tstLin,:trnMLP,:tstMLP], xlabel="Epochs", ylabel="Error")  

In [ ]:
Knet.gc() # to save gpu memory

## MLP with L1 regularization

In [ ]:
# Define new loss functions for L1 and L2 regularization
using Knet: nll
nll1(m::MLP,x,y; λ=0, o...) = nll(m,x,y; o...) + λ * sum(sum(abs, l.w) for l in m.layers)
nll2(m::MLP,x,y; λ=0, o...) = nll(m,x,y; o...) + λ * sum(sum(abs2,l.w) for l in m.layers)

In [ ]:
@time mlp3 = trainresults("mlp3.jld2", MLP(784,64,10), dtrn, dtst; loss=nll1, λ=4f-5); # 133s [0.0262292, 0.0792939, 0.0066, 0.0237]

In [ ]:
# overfitting less, test loss improves from 0.0808 to 0.0759
plot([mlp2.losses[1], mlp2.losses[2], mlp3.losses[1], mlp3.losses[2]], ylim=(0.0,0.15),
     labels=[:trnMLP :tstMLP :trnL1 :tstL1],xlabel="Epochs",ylabel="Loss")

In [ ]:
# however test error does not change significantly: 0.0235 -> 0.0220
plot([mlp2.errors[1], mlp2.errors[2], mlp3.errors[1], mlp3.errors[2]], ylim=(0.0,0.04),
     labels=[:trnMLP :tstMLP :trnL1 :tstL1],xlabel="Epochs",ylabel="Error")

In [ ]:
Knet.gc() # to save gpu memory

## MLP with dropout

In [ ]:
using Knet: dropout
@doc dropout

In [ ]:
# Dropout is another way to address overfitting
function (m::MLP)(x; pdrop=0)
    for (i,layer) in enumerate(m.layers)
        p = (i <= length(pdrop) ? pdrop[i] : pdrop[end])
        x = dropout(x, p)     # <-- This one line helps generalization
        x = layer(x)
        x = (layer == m.layers[end] ? x : relu.(x))
    end
    return x
end

In [ ]:
@time mlp4 = trainresults("mlp4.jld2", MLP(784,64,10), dtrn, dtst; pdrop=(0.2,0.0));  # 119s [0.0126026, 0.0648639, 0.0033, 0.0189]

In [ ]:
# overfitting less, loss results improve 0.0808 -> 0.0639
plot([mlp2.losses[1], mlp2.losses[2], mlp4.losses[1], mlp4.losses[2]], ylim=(0.0,0.15),
     labels=[:trnMLP :tstMLP :trnDrop :tstDrop],xlabel="Epochs",ylabel="Loss")

In [ ]:
# this time error also improves 0.0235 -> 0.0188
plot([mlp2.errors[1], mlp2.errors[2], mlp4.errors[1], mlp4.errors[2]], ylim=(0.0,0.04),
     labels=[:trnMLP :tstMLP :trnDrop :tstDrop],xlabel="Epochs",ylabel="Error")

In [ ]:
println([:mlperr,minimum(mlp2.errors[2]),:L1err,minimum(mlp3.errors[2]),:dropouterr,minimum(mlp4.errors[2])])

In [ ]:
println([:mlploss,minimum(mlp2.losses[2]),:L1loss,minimum(mlp3.losses[2]),:dropoutloss,minimum(mlp4.losses[2])])

In [ ]:
Knet.gc() # to save gpu memory

## MLP with larger hidden layer and dropout

In [ ]:
# The current trend is to use models with higher capacity tempered with dropout
@time mlp = trainresults("mlp.jld2", MLP(784,256,10), dtrn, dtst; pdrop=(0.2,0.0));  # 123s [0.00365709, 0.0473298, 0.000283333, 0.0141]

In [ ]:
plot([mlp4.losses[1], mlp4.losses[2], mlp.losses[1], mlp.losses[2]],ylim=(0,0.15),
    labels=[:trn64 :tst64 :trn256 :tst256],xlabel="Epochs",ylabel="Loss")

In [ ]:
# We are down to 0.015 error.
plot([mlp4.errors[1], mlp4.errors[2], mlp.errors[1], mlp.errors[2]],ylim=(0,0.04),
    labels=[:trn64 :tst64 :trn256 :tst256],xlabel="Epochs",ylabel="Error")